# Horizontal bar graph I

In [11]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
!pip install torch==1.12.0+cu113 torchvision==0.13.0+cu113 torchaudio==0.12.0 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install pyyaml==5.1
# !pip install torch== 1.11.0 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q pytesseract
!pip install transformers
!pip install datasets
!sudo apt-get install tesseract-ocr
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from transformers import LayoutLMv2FeatureExtractor, LayoutLMv2TokenizerFast, LayoutLMv2ForTokenClassification
from datasets import load_dataset
import pytesseract
from PIL import Image, ImageFont, ImageDraw
!pip install easyocr
import easyocr
from PIL import Image, ImageDraw, ImageFont

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html


ERROR: Could not find a version that satisfies the requirement detectron2 (from versions: none)
ERROR: No matching distribution found for detectron2


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
  Using cached https://download.pytorch.org/whl/cu113/torchvision-0.13.0%2Bcu113-cp310-cp310-win_amd64.whl (4.7 MB)
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.0+cpu
    Uninstalling torchvision-0.13.0+cpu:
      Successfully uninstalled torchvision-0.13.0+cpu


'sudo'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


RuntimeError: Failed to import transformers.models.layoutlmv2.modeling_layoutlmv2 because of the following error (look up to see its traceback):
[WinError 182] 운영 체제가 %1을(를) 실행할 수 없습니다. Error loading "C:\Users\spide\anaconda3\lib\site-packages\torch\lib\shm.dll" or one of its dependencies.

### input image

In [10]:
def detect_graph_orientation(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=250, maxLineGap=15)

    if lines is not None:
        vertical_lines = []
        horizontal_lines = []

        for line in lines:
            x1, y1, x2, y2 = line[0]
            if abs(x1 - x2) < 5:  # Check if the line is vertical
                vertical_lines.append(line)
            elif abs(y1 - y2) < 5:  # Check if the line is horizontal
                horizontal_lines.append(line)

        if len(vertical_lines) > len(horizontal_lines):
            return 1  # "Vertical"
        elif len(horizontal_lines) > len(vertical_lines):
            return 2  # "Horizontal"
        else:
            return 0

In [9]:
image_path = "C:/Users/spide/Downloads/한글 수평.png"
img = cv2.imread(image_path)

orientation = detect_graph_orientation(img)

reader = easyocr.Reader(['en', 'ko'], gpu=False)

result = reader.readtext(image_path)
img = Image.open(image_path)


if orientation == 1: #수직 그래프
    img_array = np.array(img)
    gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    result = reader.readtext(gray)
    for r in result:
        x, y = min(list(_[0] for _ in r[0])), min(list(_[1] for _ in r[0]))
        text_data = r[1]
    x_values = []
    y_values = []
    grouped_values = {}
    grouped_values_list=[]

    for r in result:
        x, y = min(list(_[0] for _ in r[0])), min(list(_[1] for _ in r[0]))
        x_values.append(x)
        y_values.append(y)

    grouped_indices = []  # 묶인 값들의 인덱스를 저장할 리스트

    for i, x in enumerate(x_values):
        if i in grouped_indices:  # 이미 묶인 값인 경우 건너뜁니다
            continue

        group = [result[i][1]]  # 현재 값과 묶일 값들을 저장할 리스트
        group_indices = [i]  # 현재 값과 묶일 값들의 인덱스를 저장할 리스트

        # 값의 차이가 25 이하인 값을 찾아서 묶어줍니다
        for j in range(i+1, len(x_values)):
            if abs(x_values[j] - x) <= 25:
                group.append(result[j][1])
                group_indices.append(j)


        if len(group) > 1:  # 묶인 값들이 2개 이상인 경우에만 처리합니다
            for value in group[1:]:
                grouped_values[value] = group[0]
                grouped_indices.extend(group_indices[1:])  # 묶인 값들의 인덱스를 추가합니다
            grouped_values_list.append(group)  # group 값을 grouped_values_list에 추가합니다


    for key, value in grouped_values.items():
        print(f"key: {key}, value: {value}")
elif orientation == 2: #수평 그래프
    img_array = np.array(img)
    gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    result = reader.readtext(gray)
    for r in result:
        x, y = min(list(_[0] for _ in r[0])), min(list(_[1] for _ in r[0]))

    x_values = []
    y_values = []
    grouped_values = {}
    grouped_values_list = []

    for r in result:
        x, y = min(list(_[0] for _ in r[0])), min(list(_[1] for _ in r[0]))
        x_values.append(x)
        y_values.append(y)

    grouped_indices = []  # 묶인 값들의 인덱스를 저장할 리스트

    for i, y in enumerate(y_values):
        if i in grouped_indices:  # 이미 묶인 값인 경우 건너뜁니다
            continue

        group = [result[i][1]]  # 현재 값과 묶일 값들을 저장할 리스트
        group_indices = [i]  # 현재 값과 묶일 값들의 인덱스를 저장할 리스트

        # 값의 차이가 25 이하인 값을 찾아서 묶어줍니다
        for j in range(i+1, len(y_values)):
            if abs(y_values[j] - y) <= 25:
                group.append(result[j][1])
                group_indices.append(j)

        if len(group) > 1:  # 묶인 값들이 2개 이상인 경우에만 처리합니다
            for value in group[1:]:
                grouped_values[value] = group[0]
                grouped_indices.extend(group_indices[1:])  # 묶인 값들의 인덱스를 추가합니다
            grouped_values_list.append(group)  # group 값을 grouped_values_list에 추가합니다


    for key, value in grouped_values.items():
        print(f"value: {value}, key: {key}")

# else:

#   재민님께서 넣으실 코드


error: OpenCV(4.5.4) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
